# Previsões do Modelo Global x Previsões de Vendas Vibra

# Setup Inicial

## Imporação das bibliotecas

In [1]:
# Manipulação de dados
import numpy as np
import pandas as pd
import json

# Manipulação de datas
from time import strftime
from datetime import datetime
import pytz

# alertas e mensagens de execução
import logging
import warnings

# Visualização
import matplotlib.pyplot as plt
import plotly
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from plotly.graph_objs import *

# Prophet para previsão de series de tempo
from prophet import Prophet


## Definições Gerais

In [2]:
# usado para remover as mensagens de output de processamento dos modelos prophet.
cmdstanpy_logger = logging.getLogger("cmdstanpy")
cmdstanpy_logger.disabled = True

In [3]:
# definção dos diretórios relativos

# replace with your container name
blob_container_name = 'general' 

# replace with your relative folder path
blob_relative_path_raw = 'raw/mercado_potencial/'
blob_relative_path_enriched = 'enriched/mercado_potencial/'


# replace with your linked service name
linked_service_raw = 'LS_ADLS_RAW_01'
linked_service_enriched = 'LS_ADLS_ENRICHED_01'

ls_raw = mssparkutils.credentials.getPropertiesAll(linked_service_raw)
ls_enriched = mssparkutils.credentials.getPropertiesAll(linked_service_enriched)

converter_dic_raw = json.loads(ls_raw)
converter_dic_enriched = json.loads(ls_enriched)

end_point_raw = (converter_dic_raw['Endpoint'].split("/"))[2]
end_point_enriched = (converter_dic_enriched['Endpoint'].split("/"))[2]


dir_raw = 'abfss://%s@%s/%s' % (blob_container_name, end_point_raw, blob_relative_path_raw)
dir_enriched = 'abfss://%s@%s/%s' % (blob_container_name, end_point_enriched, blob_relative_path_enriched)


## Funções auxiliares

In [4]:
def modelo_final (dados, periodos = 12):
    r"""Ajusta modelo prophet de serie temporal.

        Define, ajusta e prevê modelo de serie temporal baseado na biblioteca prophet.
        Modelo com a possibilidade de inclusão de variáveis regressoras. 
        
    Parameters
    ----------
    dados: pd.DataFrame 
        Dataframe com variáveis exigidas para a criação do modelo prophet.
        No dataframe deve conter, obrigatoriamente, coluna 'ds': datatime da série e 
        'y': variável alvo do modelo.
        As variáveis regressoras, quando existire, devem estar presentes no mesmo dataframe.
    periodos: int
        Períodos, em meses, a serem previstos pelo modelo.
        Por padrão, doi feinido 12 meses a serem previstos.
    
    Returns
    -------
    previsao: pd.DataFrame
        Dataframe com os resultados do predict model adcionada das variáveis de input do modelo.
    modelo: prophet model
        Modelo prophet ajustado (fitted) 
    """

    regressores = dados.columns.to_list()[2:]       # os 2 primeiros são DataTime e o Y da serie temporal

    lst_results = ['ds','yhat','yhat_lower', 'yhat_upper',
                   'trend', 'trend_lower', 'trend_upper',
                   'yearly', 'yearly_lower', 'yearly_upper']
    
    modelo = Prophet(seasonality_mode='multiplicative',
                     yearly_seasonality=True, 
                     weekly_seasonality=False,
                     daily_seasonality=False,
                     changepoint_range=0.8,
                     changepoint_prior_scale=0.05)

    for i in range(len(regressores)):               # passeia por cada lista agregativa
        modelo.add_regressor(regressores[i], standardize=False)
                
    modelo.fit(dados)

    futuro = modelo.make_future_dataframe(periods=periodos, freq='MS') #MS início do mês 
    futuro = pd.merge(futuro, dados, on='ds', how='outer')
    futuro = futuro.fillna(method='ffill')
    
    previsao = modelo.predict(futuro)
    previsao = pd.merge(previsao[lst_results], dados, on='ds',how='outer') # checar essa função

    return(previsao, modelo)

In [5]:
def process_modelos (dados, lista):
    r"""Processa o modelo e prepara os resultados para salvaguarda.

        Auxilia no processamento em lote de todos os modelos definidos. 
        
    Parameters
    ----------
    dt: pd.DataFrame
        Dataframe com os dados a serem processados
    lista: lista de strings 
        Lista com o nome das váriaveis escolhidas para o modelo    
    Returns
    -------
    previsao: pd.DataFrame
        Dataframe com os resultados do predict model adcionada das variáveis de input do modelo.
        Nele a variável alvo do modelo "y" retoma o nome original do dataset, com o prefixo 'y_'
        deixando clara que as demais informações 'yhat', 'trend', 'yearly' e seus intervalos de confiança
        foram obtidos por meio do 'y_' e demais regressoras do dataset. 
    """
    data = dados[lista]
    data.reset_index(drop=True,inplace=True)
    #data.columns = data.columns.str.replace('data', 'ds')
    data.columns = data.columns.str.replace(lista[1], 'y')
    
    previsao, modelo  = modelo_final(data)
    previsao.insert(1, 'y_'+lista[1], previsao['y'])
    previsao.drop('y',axis=1,inplace=True)

    return(previsao)

In [6]:
def ajustar_data(df):
    """Cria data a partir de dados de ano e mês separados.

        Ajusta e cria coluna em dataframe a partir de informações de 
        ano e mês separadas e em formato string. 
        
    Parâmetros
    ----------
    df: pd.DdataFrame
        Dataframe contendo as colunas 'Ano de MESANO' e 'Num mês'
    
    Retorno
    -------
    df: pd.DdataFrame
        Dataframe contendo a coluna 'Data' em formato datatime
    """

    df['Data'] = pd.to_datetime(['-'.join(i) for i in zip(df['Ano de MESANO'].map(str),df['Num mês'].map(str))],format='%Y-%m')
    df.sort_values(by='Data', inplace=True)
    df = df.set_index('Data', drop=True)
    return (df)

# Ajuste do Modelo Global para TRR e para Consumidor Final

## Resgate dos dados a serem usados nos modelos

In [7]:
dinamica   = pd.read_parquet(dir_enriched + 'volumetria/vol04_dinamica_diesel.parquet',
                                storage_options = {'linked_service' : linked_service_enriched})
dinamica_par   = pd.read_parquet(dir_enriched + 'volumetria/vol04_dinamica_diesel_parcial.parquet',
                                storage_options = {'linked_service' : linked_service_enriched})
cons_final = pd.read_parquet(dir_enriched + 'volumetria/vol04_metricas_global_ConsFinal.parquet',
                                storage_options = {'linked_service' : linked_service_enriched})
trr = pd.read_parquet(dir_enriched + 'volumetria/vol04_metricas_global_TRR.parquet',
                                storage_options = {'linked_service' : linked_service_enriched})

## Lista de variáveis que serão ajustadas em seus respectivos modelos

In [8]:
Best_Reg_CF = cons_final[cons_final.MAPE == cons_final.MAPE.min()].iloc[0,:][1].tolist()
lista_CF = dinamica.columns[0:2].tolist() # usando a variável de Volume
lista_CF.extend(Best_Reg_CF)

In [9]:
Best_Reg_TRR = trr[trr.MAPE == trr.MAPE.min()].iloc[0,:][1].tolist()
lista_TRR = dinamica.columns[[0,3]].tolist() # usando a variável de volume
lista_TRR.extend(Best_Reg_TRR)

## Definindo Variáveis dos Modelos

In [10]:
dict_var = {
            'ConsFinal':  lista_CF,
            'TRR'      :  lista_TRR,
}

## Processa e salva todos os resultados dos modelos

In [11]:
for i in range(len(dict_var)):
    prev = process_modelos(dinamica_par,list(dict_var.values())[i])

    #salvando data em formato string para auxiliar nos relatórios e visualizações do Tableau
    data_str = prev['ds'].dt.strftime('%Y-%m-%d').astype('string')     
    prev.insert(1, 'data_str',data_str, True)
    if prev.columns[2] == 'y_Indice_ConsumidorFinal':
        prev = pd.merge(prev,dinamica_par[['ds','Volume_ConsumidorFinal']] , on='ds',how='outer') # checar essa função
    else:
        prev = pd.merge(prev,dinamica_par[['ds','Volume_TRR']] , on='ds',how='outer') # checar essa função

    prev.to_parquet(dir_enriched + 'volumetria/vol05_prev_global_' + 
                                     list(dict_var.keys())[i] +
                                     '.parquet', 
                      storage_options = {'linked_service':linked_service_enriched})

# Exposição dos Resultados

## Resgatando dados das Previsões Globais TRR e Consumidor Final

In [12]:
cf = pd.read_parquet(dir_enriched + 'volumetria/vol05_prev_global_ConsFinal.parquet',
                        storage_options = {'linked_service' : linked_service_enriched})
trr = pd.read_parquet(dir_enriched + 'volumetria/vol05_prev_global_TRR.parquet',
                        storage_options = {'linked_service' : linked_service_enriched})

In [13]:
# filtrando dados a partir de 2016
cf = cf[cf['ds']>='2016']
trr = trr[trr['ds']>='2016']

## Resgatando dados das Previsões de Vendas Vibra - Vegetativo

In [14]:
prev_vibra  = pd.read_parquet(dir_enriched + 'volumetria/vol01_prev_vegetativo.parquet',
                                storage_options = {'linked_service' : linked_service_enriched})



In [15]:
prev_cf  = prev_vibra[prev_vibra['Atv_Economica']=='CONSUMIDOR FINAL']
prev_trr = prev_vibra[prev_vibra['Atv_Economica']=='TRR']

In [16]:
prev_cf  = prev_cf[:-1] # remover o último mês  - normalmente mês corrente - dados imcompletos
prev_trr = prev_trr[:-1]

## Gráfico Comparativo

In [17]:
# informações
# primeiro dado nan - mes ainda a ser disponibilizado o pib
pri = dinamica.loc[pd.isna(dinamica['PIB_Exportacao']), :] #primeiro nan
fim = dinamica['ds'].max()
#pri = pri.strftime('%Y-%m')

In [18]:

fig = make_subplots(rows=2, cols=1,
                    y_title='Volume (m³)',
                    subplot_titles=('Consumidor Final', 'TRR',),
                    shared_yaxes=False,
                    vertical_spacing=0.1 )

# grafico 01
fig.add_trace(go.Scatter(
            x = cf['ds'],
            y = cf['yhat'],
            line = dict(color='red', width=3),
            name = 'Global',
            legendgroup = 'Global',
            showlegend=True),
    row=1, col=1)

fig.add_trace(go.Scatter(
            x = prev_cf['ds'],
            y = prev_cf['yhat'],
            line = {'color': 'green'},
            name = 'Vegetativo VIBRA',
            legendgroup = 'Veg_VIBRA',
            showlegend=True),
    row=1, col=1)

# grafico 02
fig.add_trace(go.Scatter(
            x = trr['ds'],
            y = trr['yhat'],
            line = dict(color='red', width=3),
            name = 'Global',
            legendgroup = 'Global',
            showlegend=False),
    row=2, col=1)

fig.add_trace(go.Scatter(
            x = prev_trr['ds'],
            y = prev_trr['yhat'],
            line = {'color': 'green'},
            name = 'Vegetativo VIBRA',
            legendgroup = 'Veg_VIBRA',
            showlegend=False),
    row=2, col=1)

if len(pri)>0:
    fig.add_annotation(
            text= ('<b>Nota: </b> <br>De ' + pri.index[0].strftime('%b de %Y') +' a ' + fim.strftime('%b de %Y')) + 
            (' algumas regressoras tiveram imputação de dado por meio de médias móveis.<br>Pois, estas ainda não foram disponibilizadas pelas suas fontes geradoras.'),
            showarrow=False,
            font=dict(size=12),
            align="left",
            x=0.0,
            y=-0.2,
            xref='paper',
            yref='paper',
            xanchor='left',
            yanchor='bottom',
            xshift=-1,
            yshift=-5)

fig.add_annotation(
            showarrow=False,
            text='Gerado em ' + datetime.now().strftime("%d %b %Y às %H:%M:%S"),
            font=dict(size=12),
            align="right",
            x=1.18,
            y=-0.2,
            xref='paper',
            yref='paper',
            xanchor='right',
            yanchor='bottom',)
    
fig.update_xaxes(matches='x', rangeslider_visible=True, rangeslider_thickness = 0.05)
fig.update_xaxes(rangeslider= {'visible':False}, row=1, col=1)

fig.update_layout(
            title = 'Previsão de Vendas Diesel' + '<br><sup>'+ 'jan/2016 - nov/2023' + '</sup>',
            #xaxis_title = "Período",
            #yaxis_title = "y_label",
            legend_title = "Legenda",
            font=dict(
                family = "Courier New, monospace",
                size = 14,
                color = "royalblue"),
            height=900
)
fig.show()
fig = plotly.offline.plot(fig, output_type='div')
displayHTML(fig)


# Acesso à Documentação

In [19]:
help(modelo_final)

In [20]:
help(process_modelos)

In [21]:
help(ajustar_data)